# Instalasi Dependensi
Program ini membutuhkan dependensi sebagai berikut untuk dapat bekerja:
1. Pytorch
2. Beautifulsoup4
3. Pandas
4. Request
5. Numpy
6. Transformers

In [3]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# Instansiasi Model

In [11]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [17]:
tokens = tokenizer.encode('I like this place very much', return_tensors='pt')

In [18]:
result = model(tokens)

In [19]:
result.logits

tensor([[-2.8710, -2.6713, -0.3257,  2.2029,  2.8135]],
       grad_fn=<AddmmBackward0>)

In [20]:
int(torch.argmax(result.logits))+1

5

# Data Collection

Data text yang diambil oleh program ini didapatkan dari website https://www.yelp.com/biz/chicago-2-barbershop-san-francisco?osq=Black+Owned+Barber+Shops sebagai bentuk ulasan Toko Babershop yang ada di Amerika.

In [21]:
r = requests.get('https://www.yelp.com/biz/chicago-2-barbershop-san-francisco?osq=Black+Owned+Barber+Shops')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [33]:
for review in reviews:
    print(f'{review}\n')

Being from NYC I take my cuts really seriously and was nervous about how the guys in the bay cut hair out here. Jabari T laced me with a fresh tape-er/tape-up. He is very professional, very meticulous, very gentle with the clippers ( I have sensitive skin), and takes direction well. The shop is black-owned and gives a classic barbershop vibe that just feels right. I highly recommend this place and Jabari if you are ever in the bay area.

Recommend  by a friend   Great  Barber Tamura !!! will come back definitely  will bring my boys  and the best cut and very detail  professional  glad I found this guy worth  it

I was excited to go here, I thought it was an institution, but it was probably the worst haircut I've ever had. Never again!

I'm not one to give reviews but this was an experience to remember. I got an appointment for a beard groom for the same day as I had some spare time and my usual barber was not available.Before sitting down in the chair I explicitly told the barber that 

# Melakuan Scoring

In [36]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [37]:
df.head()

,review
0,Being from NYC I take my cuts really seriously...
1,Recommend by a friend Great Barber Tamura ...
2,"I was excited to go here, I thought it was an ..."
3,I'm not one to give reviews but this was an ex...
4,That boy Temur gave me the cleanest pompadour ...


In [40]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [42]:
sentiment_score(df['review'].iloc[1])

5

In [43]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [44]:
df

,review,sentiment
0,Being from NYC I take my cuts really seriously...,5
1,Recommend by a friend Great Barber Tamura ...,5
2,"I was excited to go here, I thought it was an ...",1
3,I'm not one to give reviews but this was an ex...,4
4,That boy Temur gave me the cleanest pompadour ...,5
5,"Schedule an appointment with Temur! Friendly,...",5
6,Temur is very precise with fades and any othe...,5
7,Temur make one of the best fresh cuts! Definit...,5
8,"Joe is awesome, both his attention to detail a...",5
9,Man the vibes was great! Loved the music. I we...,5


# Export Hasil Dataframe ke Bentuk CSV

In [46]:
df.to_csv('sentiment.csv')